In [1]:
# https://github.com/kuangliu/pytorch-cifar/blob/master/models/vgg.py

%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import torchvision
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import os

import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, FastICA

from torchvision.transforms import transforms
from sklearn.metrics import roc_auc_score

np.random.seed(252525)
torch.manual_seed(252525)

from data.mvtec import *
from models.vgg import *

from models.RotNet import *

MVTEC_PATH="../../../../nas-files/mvtec"

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

shape = (3, 224, 224)
start_epoch = 0
bs = 24

log = []

for r in range(0,5):

    for clazz in range(0,15):

        randomRot = RandomRot()

        # Data
        transform_train = transforms.Compose([
             transforms.RandomCrop(shape[1], padding=4),
             transforms.RandomHorizontalFlip(),
             randomRot,
             transforms.ToTensor(),
             transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        transform_test = transforms.Compose([
            randomRot,
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        print('==> Preparing data..')   
        X_, X_valid_, X_test_, X_labels_, T = dataloader(clazz, P=224, s=224, label_per_patch=False, load_size=256, crop_size=224, normalize=False, MVTEC_PATH=MVTEC_PATH )

        X_torch = torch.from_numpy( X_.reshape((len(X_), *shape)))
        X_valid_torch = torch.from_numpy( X_valid_.reshape((len(X_valid_), *shape)))
        X_test_torch = torch.from_numpy( X_test_.reshape((len(X_test_), *shape)))
        X_comb_test_torch = torch.cat([X_valid_torch, X_test_torch], axis=0) 
        X_comb_test_labels= torch.cat([torch.zeros(len(X_valid_torch)).int(), torch.ones(len(X_test_torch)).int()])

        trainset = Dataset(X_torch, torch.ones(len(X_torch)).int(), transform_train, randomRot)       
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=bs, shuffle=True)

        testset =  Dataset(torch.cat([X_valid_torch, X_test_torch], axis=0), torch.cat([torch.zeros(len(X_valid_torch)).int(), torch.ones(len(X_test_torch)).int()]), transform_test, randomRot)
        testloader = torch.utils.data.DataLoader(testset, batch_size=bs, shuffle=False)

        ensemble = []
        for ensemble_idx in range(1):

            print('==> Building model..')
            net = VGG('VGG25', True)
            net = net.to(device)
            ensemble.append(net)

            criterion = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

            # Training
            def train(epoch):
                print('\nEpoch: %d' % epoch)
                net.train()
                train_loss = 0
                correct = 0
                total = 0
                for batch_idx, (inputs, targets) in enumerate(trainloader):
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = net(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item()
                    _, predicted = outputs.max(1)
                    total += targets.size(0)
                    correct += predicted.eq(targets).sum().item()

                print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                             % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


            def test(epoch):
                global best_acc
                net.eval()
                test_loss = 0
                correct = 0
                total = 0
                with torch.no_grad():
                    for batch_idx, (inputs, targets) in enumerate(testloader):
                        inputs, targets = inputs.to(device), targets.to(device)
                        outputs = net(inputs)
                        loss = criterion(outputs, targets)

                        test_loss += loss.item()
                        _, predicted = outputs.max(1)
                        total += targets.size(0)
                        correct += predicted.eq(targets).sum().item()

                    print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                                 % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

                if False:
                    # Save checkpoint.
                    acc = 100.*correct/total
                    if acc > best_acc:
                        print('Saving..')
                        state = {
                            'net': net.state_dict(),
                            'acc': acc,
                            'epoch': epoch,
                        }
                        if not os.path.isdir('checkpoint'):
                            os.mkdir('checkpoint')
                        torch.save(state, './checkpoint/ckpt.pth')
                        best_acc = acc

            for epoch in range(start_epoch, start_epoch+30):
                train(epoch)
                test(epoch)
                scheduler.step()

        #
        # TESTTIME
        #

        logits, labels = [], []

        for rot in range(4):

            trans = transforms.Compose([
                RandomRot(rot),
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])

            testset = Dataset(X_comb_test_torch, X_comb_test_labels, trans, None)
            testloader = torch.utils.data.DataLoader(testset, batch_size=bs, shuffle=False)

            ## LOGITS
            logits_, labels = compute_logits(testloader)

            logits.append(logits_)

        binary_labels = labels.cpu()
        logits = torch.stack(logits)

        scores = compute_scores(logits).cpu().numpy()
        auc_logit = roc_auc_score(binary_labels, scores)

        log.append(pd.DataFrame(np.asarray([r, clazz, auc_logit])[:, None].T, columns=["run", "class", "auc"])) 

        display(log[-1])

==> Preparing data..
==> Building model..

Epoch: 0
8 9 Loss: 1.612 | Acc: 31.100% (65/209)
3 4 Loss: 1.381 | Acc: 25.301% (21/83)

Epoch: 1
8 9 Loss: 0.895 | Acc: 60.766% (127/209)
3 4 Loss: 1.394 | Acc: 20.482% (17/83)

Epoch: 2
8 9 Loss: 0.190 | Acc: 97.608% (204/209)
3 4 Loss: 1.476 | Acc: 33.735% (28/83)

Epoch: 3
8 9 Loss: 0.028 | Acc: 99.522% (208/209)
3 4 Loss: 0.970 | Acc: 50.602% (42/83)

Epoch: 4
8 9 Loss: 0.011 | Acc: 100.000% (209/209)
3 4 Loss: 0.145 | Acc: 93.976% (78/83)

Epoch: 5
8 9 Loss: 0.003 | Acc: 100.000% (209/209)
3 4 Loss: 0.029 | Acc: 100.000% (83/83)

Epoch: 6
8 9 Loss: 0.001 | Acc: 100.000% (209/209)
3 4 Loss: 0.059 | Acc: 97.590% (81/83)

Epoch: 7
8 9 Loss: 0.000 | Acc: 100.000% (209/209)
3 4 Loss: 0.041 | Acc: 98.795% (82/83)

Epoch: 8
8 9 Loss: 0.000 | Acc: 100.000% (209/209)
3 4 Loss: 0.008 | Acc: 100.000% (83/83)

Epoch: 9
8 9 Loss: 0.000 | Acc: 100.000% (209/209)
3 4 Loss: 0.059 | Acc: 96.386% (80/83)

Epoch: 10
8 9 Loss: 0.000 | Acc: 100.000% (209/209

,run,class,auc
0,1.0,0.0,0.731349


==> Preparing data..
==> Building model..

Epoch: 0
11 12 Loss: 1.539 | Acc: 36.429% (102/280)
4 5 Loss: 1.385 | Acc: 28.205% (33/117)

Epoch: 1
11 12 Loss: 0.847 | Acc: 57.143% (160/280)
4 5 Loss: 1.443 | Acc: 21.368% (25/117)

Epoch: 2
11 12 Loss: 0.599 | Acc: 71.786% (201/280)
4 5 Loss: 1.988 | Acc: 25.641% (30/117)

Epoch: 3
11 12 Loss: 0.503 | Acc: 76.429% (214/280)
4 5 Loss: 4.517 | Acc: 30.769% (36/117)

Epoch: 4
11 12 Loss: 0.452 | Acc: 80.714% (226/280)
4 5 Loss: 4.283 | Acc: 36.752% (43/117)

Epoch: 5
11 12 Loss: 0.185 | Acc: 94.286% (264/280)
4 5 Loss: 2.610 | Acc: 59.829% (70/117)

Epoch: 6
11 12 Loss: 0.396 | Acc: 85.714% (240/280)
4 5 Loss: 9.207 | Acc: 29.060% (34/117)

Epoch: 7
11 12 Loss: 0.227 | Acc: 91.786% (257/280)
4 5 Loss: 3.593 | Acc: 43.590% (51/117)

Epoch: 8
11 12 Loss: 0.181 | Acc: 93.571% (262/280)
4 5 Loss: 7.655 | Acc: 52.137% (61/117)

Epoch: 9
11 12 Loss: 0.047 | Acc: 98.214% (275/280)
4 5 Loss: 3.636 | Acc: 51.282% (60/117)

Epoch: 10
11 12 Loss: 0.187

,run,class,auc
0,1.0,1.0,0.542937


==> Preparing data..
==> Building model..

Epoch: 0
10 11 Loss: 1.305 | Acc: 44.490% (109/245)
5 6 Loss: 1.377 | Acc: 27.419% (34/124)

Epoch: 1
10 11 Loss: 1.021 | Acc: 55.510% (136/245)
5 6 Loss: 1.371 | Acc: 25.806% (32/124)

Epoch: 2
10 11 Loss: 0.916 | Acc: 67.347% (165/245)
5 6 Loss: 0.876 | Acc: 64.516% (80/124)

Epoch: 3
10 11 Loss: 0.589 | Acc: 77.551% (190/245)
5 6 Loss: 0.831 | Acc: 58.871% (73/124)

Epoch: 4
10 11 Loss: 0.336 | Acc: 84.898% (208/245)
5 6 Loss: 0.266 | Acc: 91.129% (113/124)

Epoch: 5
10 11 Loss: 0.195 | Acc: 94.694% (232/245)
5 6 Loss: 0.838 | Acc: 69.355% (86/124)

Epoch: 6
10 11 Loss: 0.436 | Acc: 87.347% (214/245)
5 6 Loss: 2.549 | Acc: 46.774% (58/124)

Epoch: 7
10 11 Loss: 0.253 | Acc: 91.837% (225/245)
5 6 Loss: 0.929 | Acc: 58.871% (73/124)

Epoch: 8
10 11 Loss: 0.167 | Acc: 93.878% (230/245)
5 6 Loss: 1.004 | Acc: 70.161% (87/124)

Epoch: 9
10 11 Loss: 0.135 | Acc: 95.510% (234/245)
5 6 Loss: 0.184 | Acc: 94.355% (117/124)

Epoch: 10
10 11 Loss: 0.1

...

,run,class,auc
0,4.0,12.0,0.80694


==> Preparing data..
==> Building model..

Epoch: 0
13 14 Loss: 1.645 | Acc: 30.625% (98/320)
6 7 Loss: 1.395 | Acc: 23.750% (38/160)

Epoch: 1
13 14 Loss: 1.475 | Acc: 32.188% (103/320)
6 7 Loss: 1.403 | Acc: 30.625% (49/160)

Epoch: 2
13 14 Loss: 1.250 | Acc: 44.688% (143/320)
6 7 Loss: 1.744 | Acc: 30.625% (49/160)

Epoch: 3
13 14 Loss: 1.128 | Acc: 46.875% (150/320)
6 7 Loss: 0.989 | Acc: 49.375% (79/160)

Epoch: 4
13 14 Loss: 0.784 | Acc: 62.812% (201/320)
6 7 Loss: 1.008 | Acc: 60.000% (96/160)

Epoch: 5
13 14 Loss: 0.761 | Acc: 68.438% (219/320)
6 7 Loss: 0.627 | Acc: 75.000% (120/160)

Epoch: 6
13 14 Loss: 0.464 | Acc: 81.562% (261/320)
6 7 Loss: 0.808 | Acc: 65.000% (104/160)

Epoch: 7
13 14 Loss: 0.481 | Acc: 84.062% (269/320)
6 7 Loss: 0.489 | Acc: 81.250% (130/160)

Epoch: 8
13 14 Loss: 0.251 | Acc: 91.250% (292/320)
6 7 Loss: 0.350 | Acc: 82.500% (132/160)

Epoch: 9
13 14 Loss: 0.157 | Acc: 94.688% (303/320)
6 7 Loss: 0.826 | Acc: 65.625% (105/160)

Epoch: 10
13 14 Loss: 0

,run,class,auc
0,4.0,13.0,0.469154


==> Preparing data..
==> Building model..

Epoch: 0
8 9 Loss: 0.359 | Acc: 83.568% (178/213)
4 5 Loss: 1.388 | Acc: 24.000% (24/100)

Epoch: 1
8 9 Loss: 0.015 | Acc: 99.531% (212/213)
4 5 Loss: 1.430 | Acc: 24.000% (24/100)

Epoch: 2
8 9 Loss: 0.003 | Acc: 100.000% (213/213)
4 5 Loss: 1.448 | Acc: 25.000% (25/100)

Epoch: 3
8 9 Loss: 0.002 | Acc: 100.000% (213/213)
4 5 Loss: 1.174 | Acc: 57.000% (57/100)

Epoch: 4
8 9 Loss: 0.001 | Acc: 100.000% (213/213)
4 5 Loss: 0.237 | Acc: 92.000% (92/100)

Epoch: 5
8 9 Loss: 0.000 | Acc: 100.000% (213/213)
4 5 Loss: 0.204 | Acc: 95.000% (95/100)

Epoch: 6
8 9 Loss: 0.000 | Acc: 100.000% (213/213)
4 5 Loss: 0.170 | Acc: 97.000% (97/100)

Epoch: 7
8 9 Loss: 0.000 | Acc: 100.000% (213/213)
4 5 Loss: 0.175 | Acc: 97.000% (97/100)

Epoch: 8
8 9 Loss: 0.000 | Acc: 100.000% (213/213)
4 5 Loss: 0.203 | Acc: 95.000% (95/100)

Epoch: 9
8 9 Loss: 0.000 | Acc: 100.000% (213/213)
4 5 Loss: 0.207 | Acc: 96.000% (96/100)

Epoch: 10
8 9 Loss: 0.000 | Acc: 100.00

,run,class,auc
0,4.0,14.0,0.835417


In [12]:
log_ = pd.concat(log)

for clazz in range(15):
    auc = log_[ (log_["class"] == clazz) ].groupby("run").mean()["auc"]
    print(f"class={MVTEC.CLASSES[clazz]:10s}", f"\tauc: {auc.mean():.3f}", "\u00B1", f"{auc.std():.3f}" )

avg = log_[ (log_["class"] == clazz) ].groupby("run").mean()["auc"].mean()
print(avg)

class=bottle     	auc: 0.790 ± 0.037
class=carpet     	auc: 0.457 ± 0.084
class=leather    	auc: 0.396 ± 0.105
class=pill       	auc: 0.776 ± 0.078
class=tile       	auc: 0.473 ± 0.028
class=wood       	auc: 0.675 ± 0.073
class=cable      	auc: 0.669 ± 0.037
class=grid       	auc: 0.627 ± 0.107
class=toothbrush 	auc: 0.702 ± 0.107
class=zipper     	auc: 0.689 ± 0.105
class=capsule    	auc: 0.487 ± 0.091
class=hazelnut   	auc: 0.684 ± 0.058
class=metal_nut  	auc: 0.712 ± 0.058
class=screw      	auc: 0.472 ± 0.062
class=transistor 	auc: 0.824 ± 0.022
0.8238333333333333
